# Korrelation af breve og slag

De fleste af brevene har et sted på sig.
Jeg har konstrueret et lille datasæt med militære slag på Øst- og Vestfronten i 1. verdenskrig, og så skal vi finde ud af, hvor tæt Peter var på.

Det gør vi ved at tillade både en rumlig og tidslig margin til slagene.

75 km og 10 dage til at starte med.


In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../scripts')
import importlib
import letters_to_geopandas 
importlib.reload(letters_to_geopandas)
import battles_to_geopandas
importlib.reload(battles_to_geopandas)


<module 'battles_to_geopandas' from 'c:\\projects\\dalager\\jernkorsetbreve\\notebooks\\../scripts\\battles_to_geopandas.py'>

# Loading data

In [2]:

letters = letters_to_geopandas.create_geopandas_from_csv()	
battles = battles_to_geopandas.create_geopandas_from_battle_csv()	

# this is needed for the distance calculation to work
letters = letters.to_crs("EPSG:32634")
battles = battles.to_crs("EPSG:32634")


# Korrelation

Output er en liste af breve/slag der matcher proximity kravene



In [3]:
spatial_threshold = 100000  # in meters Example: 10 km
temporal_threshold = 10  # in days Example: 3 days
letters_battles_correlated = []

for index, letter in letters.iterrows():
    # Calculate distances from this letter to all battles
    battles['distance'] = round(battles.geometry.distance(letter.geometry),3)

    # Filter battles within spatial and temporal proximity threshold
    battles['time_diff'] = (battles['date'] - letter['date']).abs()

    battles['time_diff'] = battles.apply(lambda battle: 
                                         0 if battle['date'] <= letter['date'] <= battle['enddate']
                                         else min(abs((letter['date'] - battle['date']).days), abs((letter['date'] - battle['enddate']).days)), axis=1)

    battles_close_to_letters = battles[(battles['distance'] <= spatial_threshold) & (battles['time_diff'] <= temporal_threshold)]

    for idx, battle in battles_close_to_letters.iterrows():
        letters_battles_correlated.append({
            'letter_date': letter['date'],
            'letter_place': letter['place'],
            'battle': battle['battle'],
            'battle_start': battle['date'],
            'battle_end': battle['enddate'],
            'distance_km': round(battle['distance']/1000),
            'time_diff': battle['time_diff']
        })

# Convert to DataFrame for analysis
letters_battles_correlated = pd.DataFrame(letters_battles_correlated)
pd.set_option('display.max_rows', 100)
letters_battles_correlated


,letter_date,letter_place,battle,battle_start,battle_end,distance_km,time_diff
0,1914-08-17,Lissen (Suwalki),Battle of Stalluponen,1914-08-17,1914-08-17,74,0
1,1914-08-18,Lissen (Suwalki),Battle of Stalluponen,1914-08-17,1914-08-17,74,1
2,1914-08-19,Johannisburg (Pisz),Battle of Tannenberg,1914-08-23,1914-08-30,89,4
3,1914-08-23,Olsztyn,Battle of Tannenberg,1914-08-23,1914-08-30,1,0
4,1914-08-23,Olsztyn,First Battle of the Masurian Lakes,1914-09-02,1914-09-16,71,10
...,...,...,...,...,...,...,...
56,1918-05-24,Laon,Spring Offensive,1918-03-21,1918-07-18,96,0
57,1918-05-26,Laon,Spring Offensive,1918-03-21,1918-07-18,96,0
58,1918-05-26,Laon,Spring Offensive,1918-03-21,1918-07-18,96,0
59,1918-05-27,Laon,Spring Offensive,1918-03-21,1918-07-18,96,0
